In [1]:
#OpenAI
%pip install openai==0.28.1
 
# LangChain package
%pip install -U langchain
 
#This is needed in order to for OpenAIEmbeddings
%pip install tiktoken
 
# %pip install arxiv
 
# %pip install langchain openai google-search-results
 
# %pip install wikipedia
 
# %pip install numexpr  #LLMMathChain
 
 
#%pip install PyPDF2
 
%pip install faiss-cpu
 
%pip install chromadb==0.3.29

!pip install pypdf==3.12.1

StatementMeta(5e7fc659-e1d1-4302-b80f-a5f1f8c968f2, 525, 15, Finished, Available)

[2024-01-30T11:34:59.526392] post artifact meta request successfully.
[2024-01-30T11:34:59.648331] post artifact meta request successfully.
Successfully add logs to artifact services.
Start to mount artifact container with mount point /AmlJobLogs/dcid.092a2b97-655e-470a-b64a-a070e2b0691b
Successfully mount artifact container to path:  /synfs/notebook/AmlJobLogs/dcid.092a2b97-655e-470a-b64a-a070e2b0691b
Starting log upload threads


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 KB 17.1 MB/s eta 0:00:00
You should consider upgrading via the '/nfs4/pyenv-aeaa79eb-ee00-4e5f-9c09-bdcef9719555/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 KB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 KB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 KB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 KB 30.1 MB/s 

     |████████████████████████████████| 254 kB 19.6 MB/s eta 0:00:01



## 1. Import the libraries.

In [2]:
import os
import openai

openai.api_type = "azure"
openai.api_version = "2023-07-01-preview" 
openai.api_base = "****"  # Your Azure OpenAI resource's endpoint value.
openai.api_key = "*****"

DEPLOYMENT_NAME = "gpt-35-turbo-inter02"

StatementMeta(5e7fc659-e1d1-4302-b80f-a5f1f8c968f2, 525, 21, Finished, Available)

[2024-01-30T11:39:05.749070] get artifact sas uri successfully. origin is ExperimentRun artifact path is logs/azureml/driver/stdout
Successfully add logs to artifact services.
Start to mount artifact container with mount point /AmlJobLogs/dcid.092a2b97-655e-470a-b64a-a070e2b0691b
Successfully mount artifact container to path:  /synfs/notebook/AmlJobLogs/dcid.092a2b97-655e-470a-b64a-a070e2b0691b
Starting log upload threads


In [4]:
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

StatementMeta(5e7fc659-e1d1-4302-b80f-a5f1f8c968f2, 525, 23, Finished, Available)

## 2. Document Loading

In [5]:
# load document
loader = PyPDFLoader("Users/rajat.chaudhari/RAGInputPaper.pdf")
documents = loader.load()
print(documents[1])

StatementMeta(5e7fc659-e1d1-4302-b80f-a5f1f8c968f2, 525, 24, Finished, Available)

page_content=" \nAll Sciences  Proceedings  \nhttp://as -proceeding.com/  5th International Conference on Applied \nEngineering and Natural Sciences  \n \nJuly 10 -12, 2023 : Konya, Turkey  \n \nhttps://www.icaens.com/  © 20 23 Published by All Sciences Proceedings  \n \n1050  \n \n \nCreating Large Language Model Applications Utilizing LangChain: A \nPrimer on Developing LLM Apps Fast  \nOguzhan Topsakal1*, and Tahi r Cetin Akinci  2  \n1Computer Science Department , Florida Polytechnic University, FL, USA  \n2WCGEC , University  of California at Riverside , CA, USA  \n*(otopsakal @floridapoly .edu) Email of the corresponding author  \n \nAbstract – This study focuses on the utilization of Large Language Models (LLMs) for the rapid \ndevelopment of applications, with a spotlight on L angChain, an open -source software library. LLMs have \nbeen rapidly adopted due to their capabilities in a range of tasks, including essay composition, code \nwriting, explanation, and debugging, with Op

## 3. Document Transformers

In [6]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

StatementMeta(5e7fc659-e1d1-4302-b80f-a5f1f8c968f2, 525, 25, Finished, Available)

## 4. Document Embedding

In [7]:
embeddings= OpenAIEmbeddings(
        deployment="text-embedding-ada-002-inter02",
        openai_api_key=openai.api_key,
        openai_api_base=openai.api_base,
        openai_api_type=openai.api_type,
        openai_api_version=openai.api_version,
        chunk_size = 1
    )

StatementMeta(5e7fc659-e1d1-4302-b80f-a5f1f8c968f2, 525, 26, Finished, Available)

/nfs4/pyenv-aeaa79eb-ee00-4e5f-9c09-bdcef9719555/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


## 5. Vector Storage

In [8]:
db = Chroma.from_documents(texts, embeddings)

StatementMeta(5e7fc659-e1d1-4302-b80f-a5f1f8c968f2, 525, 27, Finished, Available)

In [9]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})

StatementMeta(5e7fc659-e1d1-4302-b80f-a5f1f8c968f2, 525, 28, Finished, Available)

In [10]:
llm = AzureChatOpenAI(model="gpt-35-turbo",
                    deployment_name="gpt-35-turbo-inter02",
                    openai_api_key=openai.api_key,
                    openai_api_base=openai.api_base,
                    openai_api_type=openai.api_type,
                    openai_api_version=openai.api_version,
                     )

StatementMeta(5e7fc659-e1d1-4302-b80f-a5f1f8c968f2, 525, 29, Finished, Available)

/nfs4/pyenv-aeaa79eb-ee00-4e5f-9c09-bdcef9719555/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [11]:
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

StatementMeta(5e7fc659-e1d1-4302-b80f-a5f1f8c968f2, 525, 30, Finished, Available)

In [18]:
query = "What is LangChain?"
answer = qa({"query": query})
print(query)
print("\n")
print("Answer:",answer.get('result'))

StatementMeta(5e7fc659-e1d1-4302-b80f-a5f1f8c968f2, 525, 37, Finished, Available)

What is LangChain?


Answer: LangChain is a framework for developing applications that utilize large language models (LLMs). It consists of various building blocks called "chains" that combine an LLM with a prompt to perform operations on text or other data. LangChain supports different types of chains, such as Simple Sequential Chains and Sequential Chains, which can be used to create sequences of operations. The framework also includes a memory component that stores previous conversations and passes them to the LLM with the next prompt, creating the illusion of memory in chatbot systems. Overall, LangChain enables the development of LLM applications quickly and efficiently.


In [19]:
query = "What are Action Agents?"
answer = qa({"query": query})
print(query)
print("\n")
print("Answer:",answer.get('result'))

StatementMeta(5e7fc659-e1d1-4302-b80f-a5f1f8c968f2, 525, 38, Finished, Available)

What are Action Agents?


Answer: Action Agents are agents within an agent-based system that operate at a high level by receiving user input, determining the appropriate tool and its input, executing the tool, recording its output, and making decisions on subsequent steps based on the history of tool usage, inputs, and observations. These agents encapsulate the sequence of actions and interactions with the tools, allowing for flexible chains of calls to various tools based on user input.


In [20]:
query = "What are Action Agents? present you answer in points"
answer = qa({"query": query})
print(query)
print("\n")
print("Answer:",answer.get('result'))

StatementMeta(5e7fc659-e1d1-4302-b80f-a5f1f8c968f2, 525, 39, Finished, Available)

What are Action Agents? present you answer in points


Answer: Action Agents are agents that operate at a high level and make decisions based on previous actions and observations. Here are some key points about Action Agents:

1. Purpose: Action Agents are used in situations where an app needs a flexible chain of calls to Language Model Managers (LLMs) and other tools based on user input.

2. Decision-making: Action Agents determine the appropriate tool and its input based on the outputs of all previous actions.

3. Execution: Action Agents execute the selected tool, record its output as an observation, and make decisions on subsequent steps based on the history of tool usage, inputs, and observations.

4. Cycle: The cycle of receiving user input, determining the tool, executing it, recording the output, and making subsequent decisions repeats until the agent can directly respond to the user.

5. Agent Executors: Action Agents are encapsulated in agent executors that manage the sequenc

In [21]:
query = "What are limitations of LLM? present you answer in points"
answer = qa({"query": query})
print(query)
print("\n")
print("Answer:",answer.get('result'))

StatementMeta(5e7fc659-e1d1-4302-b80f-a5f1f8c968f2, 525, 40, Finished, Available)

What are limitations of LLM? present you answer in points


Answer: The limitations of Large Language Models (LLMs) are as follows:

1. Erroneous Outputs: LLMs can occasionally generate erroneous or illogical outputs, also known as hallucinations. These models may produce sentences or paragraphs that do not make sense or contain incorrect information.

2. Lack of Contextual Understanding: LLMs may struggle with understanding the context of a given prompt. They rely heavily on the preceding text to generate coherent responses, which can lead to misunderstandings or irrelevant answers if the context is not properly conveyed.

3. Sensitivity to Input Changes: LLMs can be sensitive to slight changes in the input prompt. Modifying a single word or phrase in the prompt can result in significantly different outputs, making it challenging to control or fine-tune the model's responses.

4. Lack of Common Sense Reasoning: While LLMs excel at syntactic and grammatical coherence, they often lack c

In [22]:
query = "What are the use cases mentioned in the document"
answer = qa({"query": query})
print(query)
print("\n")
print("Answer:",answer.get('result'))

StatementMeta(5e7fc659-e1d1-4302-b80f-a5f1f8c968f2, 525, 41, Finished, Available)

What are the use cases mentioned in the document


Answer: The document mentions several use cases for the LangChain framework. Some of the mentioned use cases include:
- Autonomous agents
- Chatbots
- Code understanding agents
- Extraction
- Question answering over documents
- Summarization
- Analyzing structured data


In [23]:
query = "Who is best US President?"
answer = qa({"query": query})
print(query)
print("\n")
print("Answer:",answer.get('result'))

StatementMeta(5e7fc659-e1d1-4302-b80f-a5f1f8c968f2, 525, 42, Finished, Available)

Who is best US President?


Answer: Based on the provided context, there is no information or discussion about the best US President. Therefore, I don't have the information to answer your question.


# Rajat.Chaudhari